In [2]:
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.cross_validation import train_test_split

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [108]:
DATA_PATH = '../data/'
INPUT_TRAIN = DATA_PATH+'input_train.csv'
OUTPUT_TRAIN = DATA_PATH+'output_train.csv'

INPUT_TEST = DATA_PATH+'input_train.csv'

In [109]:
input_train = pd.read_csv(INPUT_TRAIN,index_col='Id')
output_train = pd.read_csv(OUTPUT_TRAIN,sep=';',index_col='Id')

input_test = pd.read_csv(INPUT_TEST,index_col='Id')

In [110]:
ID_2014 = output_train[output_train['2014']==1].index.tolist()
ID_2015 = output_train[output_train['2015']==1].index.tolist()

In [111]:
input_train.iloc[ID_2014].shape

(53, 7)

In [112]:
input_train.shape

(19427, 7)

In [73]:
for k in range(0,10):
    input_train = pd.concat([input_train.loc[ID_2014],input_train])
    output_train = pd.concat([output_train.loc[ID_2014],output_train])

In [74]:
print(input_train.shape)
output_train.shape

(73646, 7)


(73646, 2)

In [75]:
def preprocess(dataframe,year=2014):
    X = dataframe

    X['YearsOld'] = year - X['YearConstruction']
    X = X.fillna(10000)
    X['YearsOldLastFailure'] = year - X['YearLastFailureObserved']

    X = X.drop(['YearConstruction','YearLastFailureObserved'],axis=1)

    X = pd.concat([X,pd.get_dummies(X['Feature1'])],axis=1)
    X = pd.concat([X,pd.get_dummies(X['Feature2'])],axis=1)
    X = pd.concat([X,pd.get_dummies(X['Feature4'])],axis=1)

    X = X.drop(["Feature1","Feature2","Feature4"],axis=1)
   
    
    X['Feature3'] = normalize(X['Feature3']).tolist()[0]
    X['Length'] = normalize(X['Length']).tolist()[0]
    X['YearsOld'] = normalize(X['YearsOld']).tolist()[0]
    X['YearsOldLastFailure'] = normalize(X['YearsOldLastFailure']).tolist()[0]
     
    
    
    return X

In [76]:
#preprocess(input_test)

In [77]:
def preprocess_output(dataframe,year=2014):
    return dataframe[str(year)]

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [79]:
X = preprocess(input_train)

In [80]:
output_train[output_train['2014']==1].shape

(54272, 2)

In [81]:
test = preprocess(input_test)

In [82]:
X = preprocess(input_train)
Y = preprocess_output(output_train)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

logreg = LogisticRegression(C=1e1,class_weight='balanced')
logreg.fit(X_train, y_train)


from sklearn.metrics import classification_report

y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred))

sub_1 = logreg.predict_proba(test)

             precision    recall  f1-score   support

          0       0.57      0.78      0.66      3895
          1       0.91      0.79      0.85     10835

avg / total       0.82      0.79      0.80     14730



In [113]:
for k in range(0,10):
    input_train = pd.concat([input_train.loc[ID_2015],input_train])
    output_train = pd.concat([output_train.loc[ID_2015],output_train])

In [114]:
X = preprocess(input_train,year=2015)
Y = preprocess_output(output_train,year=2015)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

logreg = LogisticRegression(C=1e5,class_weight='balanced')

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train, y_train)

from sklearn.metrics import classification_report

y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred))

sub_2 = logreg.predict_proba(test)

             precision    recall  f1-score   support

          0       0.65      0.78      0.70      5830
          1       0.87      0.78      0.82     11354

avg / total       0.79      0.78      0.78     17184



In [115]:
submission = pd.DataFrame()

In [116]:
sub_1[:,0].shape

(19427,)

In [117]:
submission['Id'] = test.index.tolist()
submission['2014'] = sub_1[:,0]
submission['2015'] = sub_2[:,0]

In [121]:
submission.to_csv('../submissions/data_augmentation_logreg.csv',index=False,decimal=';')